In [1]:
%load_ext autoreload
%autoreload 2

import cv2
import os
import numpy as np
import matplotlib
import tkinter
import matplotlib.pyplot as plt
import random
matplotlib.use('TkAgg')
%matplotlib inline 
from PIL import Image

import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
from stretch_body.robot import Robot
from scipy.spatial.transform import Rotation
from re1_utils.camera import get_cur_rs_frame
from re1_utils.objdet_utils import plot_all_boxes


**Object Detection**: From yolov7

In [2]:
import sys
sys.path.append('./yolov7/')
from yolov7.predict import load_model, preprocessing, \
    postprocessing, load_config

In [3]:
#Load config from yaml
CONFIG_PATH = './yolov7/predict_config.yaml'
config_dict = load_config(CONFIG_PATH)

#Load model, edit yolov7.pt path
model, stride, device = load_model(config_dict, '../yolov7/yolov7.pt')


[INFO] [utils.torch_utils]: YOLOR 🚀 a0bbdb1 torch 1.12.1+cu102 CPU



Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


[INFO] [utils.torch_utils]: Model Summary: 306 layers, 36905341 parameters, 36905341 gradients


Read sample img

In [4]:
#Read img0 to BGR
img0 = cv2.imread('./yolov7/inference/images/image1.jpg')
# img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
#Resize, pad
img = preprocessing(config_dict, img0, stride, device) 

#Inference
pred = model(img, augment=config_dict['augment'])[0] #Shape (1, num_preds, 85)

#nms and scale coordinates
pred = postprocessing(config_dict, pred, img0.shape, img.shape)

#xyxy, conf_score, class
print(pred, pred.shape)

[[        250          78         633         531     0.96753           0]
 [          2           5         402         527     0.95566           0]
 [        196         199         254         418     0.88187          27]
 [        256         207         299         247     0.42756          55]] (4, 6)


/home/hello-robot/anaconda3/envs/RE1/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)

plot_all_boxes(pred, img0)
# plt.imshow(img0)
# plt.show()

img = Image.fromarray(img0, 'RGB')
img.show()


In [6]:
color_frame, color, depth_frame, depth = get_cur_rs_frame()

Frames Captured


In [7]:
from re1_utils.objdet_utils import plot_all_boxes

#Resize, pad
color_img = preprocessing(config_dict, color, stride, device) 

#Inference
pred = model(color_img, augment=config_dict['augment'])[0] #Shape (1, num_preds, 85)

#nms and scale coordinates
pred = postprocessing(config_dict, pred, color.shape, color_img.shape)

#xyxy, conf_score, class
print(pred, pred.shape)
color = np.ascontiguousarray(color, dtype=np.uint8)

plot_all_boxes(pred, color)
color_img = Image.fromarray(color, 'RGB')
color_img.show()
# plt.imshow(color)
# plt.show()


[[        121         342         296         628     0.89368          56]
 [        351         398         456         595     0.88076          56]
 [        243         303         380         373     0.81538          62]
 [        221         521         250         610     0.45188          39]
 [        402         345         421         394     0.42215          39]
 [        291         375         401         398     0.36896          66]
 [        402         345         421         394     0.36451          41]] (7, 6)


In [8]:
from re1_utils.camera import get_rs_colorized_depth
colorized_depth = get_rs_colorized_depth(depth_frame=depth_frame)
depth_img = Image.fromarray(colorized_depth, 'RGB')
depth_img.show()
# plt.imshow(colorized_depth)
# plt.show()

**Testing landmark screen**

In [9]:
from re1_utils.camera import get_cur_rs_frame
color_frame, color, depth_frame, depth = get_cur_rs_frame()
color = np.ascontiguousarray(color, dtype=np.uint8)


Frames Captured


In [10]:
from POI.landmark_screen import LandmarkScreen


landmark_screen = LandmarkScreen(color_frame=color, depth_frame=depth)
landmark_screen.update_OOI(pred)
landmark_screen.show()
landmark_screen.show_OOI()

chair at ((121, 342), (296, 628)) depth 2.690000057220459 meters
chair at ((351, 398), (456, 595)) depth 3.319000244140625 meters
tv at ((243, 303), (380, 373)) depth 3.132000207901001 meters
bottle at ((221, 521), (250, 610)) depth 1.8850001096725464 meters
bottle at ((402, 345), (421, 394)) depth 3.319000244140625 meters
keyboard at ((291, 375), (401, 398)) depth 3.069000244140625 meters
cup at ((402, 345), (421, 394)) depth 3.319000244140625 meters


**Test Intrinsic Matrix**

In [11]:
#Test intrinsic matrix
from re1_utils.camera import get_rs_intrinsic_mat
intrinsic_mat = get_rs_intrinsic_mat()
print(intrinsic_mat)
inv_intrinsic_mat = np.linalg.inv(intrinsic_mat)
print(inv_intrinsic_mat)


[[     606.46           0      323.98]
 [          0       606.4      234.81]
 [          0           0           1]]
[[  0.0016489           0     -0.5342]
 [          0   0.0016491    -0.38722]
 [          0           0           1]]


In [12]:
from POI.landmark_screen import LandmarkScreen

landmark_screen.update_cam_coords(inv_intrinsic_mat)

In [13]:
#Please edit index
keyboard = landmark_screen.get_OOI()[5]
keyboard.show()
closest_bottle = landmark_screen.get_OOI()[3]
closest_bottle.show()

keyboard at ((291, 375), (401, 398)) depth 3.069000244140625 meters
bottle at ((221, 521), (250, 610)) depth 1.8850001096725464 meters


In [14]:
print(keyboard.img_coord, keyboard.cam_coord)
print(closest_bottle.img_coord, closest_bottle.cam_coord)


[        346       386.5           1] [    0.11146     0.76769       3.069]
[      235.5       565.5           1] [     -0.275      1.0279       1.885]


**Test extrinsic matrix**

In [15]:
#Test intrinsic matrix
from re1_utils.camera import get_rs_extrinsic_mat
extrinsic_mat = get_rs_extrinsic_mat(type = 'cam2base')
print(extrinsic_mat)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[[    0.99917   -0.036799    0.017461    0.012127]
 [   0.018093    0.016906    -0.99969    0.016835]
 [   0.036493     0.99918    0.017558      1.2441]
 [          0           0           0           1]]
[[    0.99917    0.018093    0.036493   -0.057821]
 [  -0.036799    0.016906     0.99918     -1.2429]
 [   0.017461    -0.99969    0.017558  -0.0052257]
 [          0           0           0           1]]


In [16]:
from POI.landmark_screen import LandmarkScreen

landmark_screen.update_base_coords(extrinsic_mat)

In [17]:
print(keyboard.cam_coord, keyboard.base_coord)
print(closest_bottle.cam_coord, closest_bottle.base_coord)

[    0.11146     0.76769       3.069] [    0.14883     -3.0362      2.0691]
[     -0.275      1.0279       1.885] [   -0.26756     -1.8552      2.2942]


In [ ]:
sys.path.append('..')

In [ ]:
from POI.object_of_interest import OOI
all_objects = []
for i in range(pred.shape[0]):
    coord = pred[i, :]
    obj = OOI(
        img_coord = np.array([(coord[2]-coord[0])/2,(coord[3]-coord[1])/2]), 
        depth = 0,
        obj_class = int(coord[-1]),
        obj_atributes = 'None', 
        bbox = ((coord[0],coord[1]),(coord[2],coord[3])),
        conf_score = coord[4],
        eid = i
    )
    all_objects.append(obj)

MEMORY

*Taken from fairo tutorial and memory module


**Memory**

Now we have setup a small object detection + deduplication pipeline.

But the robot is not storing the information of these objects in it's memory yet.

If it doesn't store this information in memory, then when you say "go to the chair", it does not know where the chair is (unless the chair is in it's field of view at that given moment).

`droidlet` provides a memory system that can store generic metadata. This memory system is used by the Dialog Parser + Task controller to do tasks utilizing context provided by information stored in memory.

The memory is backed by an SQL database, and has schemas to represent common semantic information for robots and the environment.

Let us first create the default `AgentMemory` object for our Locobot using some pre-baked and thoughtful memory schemas.

In [ ]:
sys.path.append('./memory/')


In [ ]:
from memory.sql_memory import AgentMemory
from memory.robot.loco_memory_nodes import NODELIST

SQL_SCHEMAS = [
    os.path.join(os.getcwd(), "memory", "base_memory_schema.sql"),
    os.path.join(os.getcwd(), "memory", "robot","loco_memory_schema.sql"),
]

In [ ]:
memory = AgentMemory(db_file=":memory:", schema_paths=SQL_SCHEMAS, nodelist=NODELIST)

We can see the types of nodes that can be stored inside the memory

In [ ]:
memory.nodes

Let us store the previously detected objects into memory, using this new memory system.

For this, we will use the DetectedObjectNode. A physical object is represented in memory as a DetectedObjectNode, which is thoughtfully annotated with properties such as it's color and it's detected xyz location.

As a reminder, in the previous section, we deduplicated the objects detected in the scene, and stored them in the variable previous_objects.

Now, we will store these all_objects into the memory. Let us start with storing and retreiving one object, and inspecting the results.

In [ ]:
# from memory.robot.loco_memory_nodes import Detect
from memory.robot.loco_memory_nodes import BCIDetectedObjectNode


In [ ]:
memory_ids = []
for i in range(len(all_objects)):
    memory_id = BCIDetectedObjectNode.create(memory, all_objects[i])
    memory_ids.append(memory_id)

Now, let us retreive the `DetectedObjectNode` from memory.

In [ ]:
memory.get_mem_by_id(memory_id)

The memory object is in it's raw packed form, and is not yet converted back to a dict with accessible fields.

We can access the detected objects back from memory as dicts using the `get_all` function:

In [ ]:
BCIDetectedObjectNode.get_all(memory)

Now, let us store the rest of the detected objects into memory